In [1]:
import numpy as np
from sklearn import metrics
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib


groupsizes = [4,8,16,32,64,128,256,512,1024]
coherences = [0.0,0.2,0.4,0.6,0.8,1.0]

In [2]:
alldata = pd.read_pickle('/media/recnodes/Dan_storage/20200120_coherence_data_compiled_full.pickle')

In [3]:
alldata['groupsize'] = alldata['groupsize'].astype(int)
alldata['coh'] = np.around(alldata['coh'],1)

alldata['Information'] = alldata['entropy_Ra']*-1.0 #negative entropy for information

In [4]:
responseonly = alldata.loc[alldata['syncTime'].between(np.timedelta64(270,'s'), 
                                                   np.timedelta64(300,'s')), :]
responseonly = responseonly.loc[responseonly['date'] != '20181026', :] #exclude data from this date, when fish were left in the tank overnight

In [5]:
r = responseonly[['trialID','dir','coh','groupsize','median_dRotation_cArea','entropy_Ra']]

#de-normalize by direction to give discrete answers
#multiply by 100 because silly sklearn uses max+1 in fpr
r['R'] = r['median_dRotation_cArea'] * np.around(r['dir'])*100.0
r['dir'] = np.around(r['dir'])
r = r.loc[abs(r['dir']) == 1,:] #fails when dir == 0


/home/dan/miniconda3/envs/netstuff/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/dan/miniconda3/envs/netstuff/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
def bootstrap_ROC(groupedDF, k=1000, col='R'):
    #provide data grouped by trialID
    
    # get the direction of each trial
    y_true = np.sign(groupedDF['dir'].mean())
    
    fpr_distribution = []
    tpr_distribution = []
    thresh_distribution = []
    auc_scores = []
    for idx in range(k):
        dirs = []
        resampled_trials=[]
        for trialid, trial in groupedDF:
            resample = trial.reset_index().sample(int(len(trial)/2), replace=True)
            resampled_trials.append(resample[col])
            dirs.append(resample['dir'])
            #print(resample[0:5])
        #drop_intermediate to ensure fpr, tpr, and thresholds have same length every time
        fpr, tpr, thresholds = metrics.roc_curve(np.concatenate(dirs), np.concatenate(resampled_trials), pos_label=1, drop_intermediate=False)
        auc = 'asdf'#metrics.roc_auc_score(dirs, resampled_trials)
        fpr_distribution.append(fpr)
        tpr_distribution.append(tpr)
        thresh_distribution.append(thresholds)
        auc_scores.append(auc)
    return np.stack(fpr_distribution, axis=-1),np.stack(tpr_distribution, axis=-1), np.stack(thresh_distribution, axis=-1), auc_scores

In [35]:
r['coh'] = np.around(r['coh'], 1)
rcoh = r.loc[r.coh.isin(coherences),:]
tg = rcoh.groupby('trialID')
f, t, th, auc = bootstrap_ROC(tg, k=10)

ValueError: all input arrays must have the same shape

In [33]:
l = []
m = [1,2,3,4,5]
l.append(m)
l.append(m)
l.append(m)
np.concatenate(l)

array([1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5])

In [ ]:
coherences = [0.0,0.2,0.4,0.6,0.8,1.0]

N = len(coherences)
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", 
                                             plt.cm.viridis(np.linspace(0,1,N)))
g = rcoh.groupby('coh')
for group, data in g:
    tg = data.groupby('trialID')
    fpr, tpr, thresholds, auc = bootstrap_ROC(tg, k=20)
    plt.fill_between(fpr.mean(axis=1), np.quantile(tpr,0.025, axis=1), np.quantile(tpr, 0.975, axis=1), alpha=0.4)
    plt.plot(fpr.mean(axis=1),tpr.mean(axis=1), label=group)
plt.legend()
plt.title('receiver operating characteristic (ROC)')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.plot([0,1],[0,1], 'k--')
ax.set_aspect_ratio(1)

#plt.fill_between(f.mean(axis=1), np.quantile(t,0.025, axis=1), np.quantile(t, 0.975, axis=1), alpha=0.4)
#plt.plot(f.mean(axis=1),t.mean(axis=1))
#np.quantile(t,0.25, axis=1) ==  np.quantile(t,0.75, axis=1)

In [ ]:
r['coh'] = np.around(r['coh'], 1)
rcoh = r.loc[r.coh.isin(coherences),:]
N = len(set(rcoh['coh']))

plt.rcParams["axes.prop_cycle"] = plt.cycler("color", 
                                             plt.cm.viridis(np.linspace(0,1,N)))


g = rcoh.groupby('coh')
for group, data in g:
    scores = data['R']
    y_true = np.around(data['dir'])
    fpr, tpr, thresholds = metrics.roc_curve(y_true, scores, pos_label=1)
    plt.plot(fpr, tpr, label=group)
plt.legend()
plt.title('receiver operating characteristic (ROC)')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.plot([0,1],[0,1], 'k--')

plt.savefig('/media/recnodes/Dan_storage/20200622_ROC_curve_by_coh.svg')

In [ ]:
rcoh['coh100'] = np.around(rcoh['coh']*100) #because silly AUC adds 1
g = rcoh.groupby('coh100')
xdata = []
ydata = []
for group, data in g:
    scores = data['R']
    y_true = np.sign(data['dir'])
    auc = metrics.roc_auc_score(y_true, scores)
    xdata.append(int(group))
    ydata.append(auc)
    plt.scatter(float(group/100.0), auc, label=group)

    
  
    #plt.legend()
#plt.xscale('log', basex=2)

plt.xlabel('stimulus coherence')
plt.ylabel('Area under the ROC curve')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


def func(x, a, b, c):
  #return a * np.exp(-b * x) + c
  return a * np.log(b * x) + c

from scipy.optimize import curve_fit

popt, pcov = curve_fit(func, xdata, ydata)

print(popt)
yfit = [func(X, *popt) for X in np.linspace(1,100, 100)]
#plt.plot(np.linspace(1,100, 100), yfit, 'r-',
#         label='fit: a=%5.1f, b=%5.1f, c=%5.1f' % tuple(popt))  
plt.legend()

plt.savefig('/media/recnodes/Dan_storage/20200622_ROC_AUC_by_coh.svg')

In [ ]:

N = len(set(r['groupsize']))

plt.rcParams["axes.prop_cycle"] = plt.cycler("color", 
                                             plt.cm.viridis(np.linspace(0,1,N)))


g = r.groupby('groupsize')
for group, data in g:
    scores = data['R']
    y_true = np.around(data['dir'])
    fpr, tpr, thresholds = metrics.roc_curve(y_true, scores, pos_label=1)
    plt.plot(fpr, tpr, label=group)
plt.legend()
plt.title('receiver operating characteristic (ROC)')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.plot([0,1],[0,1], 'k--')
plt.savefig('/media/recnodes/Dan_storage/20200621_ROC_curve.svg')

In [ ]:
g = r.groupby('groupsize')
xdata = []
ydata = []
for group, data in g:
    scores = data['R']
    y_true = np.sign(data['dir'])
    auc = metrics.roc_auc_score(y_true, scores)
    xdata.append(int(group))
    ydata.append(auc)
    plt.scatter(int(group), auc, label=group)

    
  
    #plt.legend()
#plt.xscale('log', basex=2)

plt.xlabel('group size')
plt.ylabel('Area under the ROC curve')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


def func(x, a, b, c):
  #return a * np.exp(-b * x) + c
  return a * np.log(b * x) + c

from scipy.optimize import curve_fit

popt, pcov = curve_fit(func, xdata, ydata)

print(popt)
yfit = [func(X, *popt) for X in np.linspace(1,1024, 100)]
plt.plot(np.linspace(1,1024, 100), yfit, 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))  
plt.legend()

plt.savefig('/media/recnodes/Dan_storage/20200621_ROC_AUC.svg')

In [ ]:


responseonly = responseonly.loc[responseonly.coh.isin(coherences),:]
cols = ['median_dRotation_cArea','std_dRotation_cArea','Information','normed']
titles = ['median','std. dev.','information', 'info/sqrt(N)']
R=1
S=0

from scipy.ndimage.filters import gaussian_filter
#from skimage.transform import resize 
from mpl_toolkits.axes_grid1 import make_axes_locatable

def heatmap(df, fig=None, ax=None, SIGMA=0, RESIZE_FACTOR=1):
    """
    pass a 2d dataframe (for example, df.groupby([col1,col2])[col3].mean().unstack() )
    
    returns a heatmap as an imshow image with optional smoothing (sigma, resizing)
    """
    if fig == None:
        fig = plt.figure()
    if ax == None:
        ax = fig.add_subplot(111)
    img = np.array(df)
    #resized = resize(img, (img.shape[0]*RESIZE_FACTOR, img.shape[1]*RESIZE_FACTOR))
    filtered_img = gaussian_filter(img, SIGMA)
    plt.sca(ax)
    #show image
    im = ax.imshow(filtered_img, origin='lower')
    #create colourbar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    return


g = responseonly.groupby('trialID').median()
g['coh'] = np.around(g['coh'], 1)
g['normed'] = g['Information']/np.sqrt(g['groupsize'])

#def plotent(S, R):
_fig = plt.figure()
for i in range(4):
    axi = _fig.add_subplot(1,4,i+1)
    heatmap(g.groupby(['groupsize','coh'])[cols[i]].mean().unstack(), 
               RESIZE_FACTOR=R, SIGMA=S,
               fig=_fig, ax=axi)
    axi.set_ylabel('Group Size')
    axi.set_xlabel('Coherence')
    axi.set_title(titles[i])
    axi.spines['top'].set_visible(False)
    axi.spines['right'].set_visible(False)
#plt.show()
#    return
plt.savefig('/media/recnodes/Dan_storage/20200621_heatmaps.svg')

In [ ]:
g['normed'] = g['Information']/g['groupsize']
g.groupby(['groupsize','coh'])['normed'].mean().unstack()